# 1. Import necessary libraries

In [ ]:
# pip install rasterio 
# pip install spectral
import matplotlib.pyplot as plt
import numpy as np
from time import time
import rasterio as rio

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from sklearn.preprocessing import minmax_scale
from sklearn import cluster
from sklearn.decomposition import PCA


## 2. Read the data

In [ ]:
# Importing the data
data_raster = rio.open('drive/MyDrive/VAE_GeoChem/Delamerian_Landsat8.tif')
# data_raster = rio.open('drive/MyDrive/VAE_GeoChem/Delamerian_ASTER.tif')

# print(data_raster.meta)

## Visualizing the data
# Reading and enhancing
data_array = data_raster.read() # reading the data
# vmin, vmax = np.nanpercentile(data_array, (5,95)) # 5-95% pixel values stretch
# Plotting the enhanced image
# fig = plt.figure(figsize=[20,20])
# plt.axis('off')
# plt.imshow(data_array[1, :, :], cmap='gray', vmin=vmin, vmax=vmax)
# plt.show()

## 3. Reshape the hyper-spectoral image (HSI)

In [ ]:
## Reshaping the input data from brc to rcb
# Creating an empty array with the same dimension and data type
imgxyb = np.empty((data_raster.height, data_raster.width, data_raster.count), data_raster.meta['dtype'])
# Looping through the bands to fill the empty array
for band in range(imgxyb.shape[2]):
    imgxyb[:,:,band] = data_raster.read(band+1)

# Reshaping the input data from rcb to samples and features
data_reshaped = imgxyb.reshape(imgxyb.shape[0]*imgxyb.shape[1], -1)
# Scaling
data_reshaped = minmax_scale(data_reshaped, feature_range=(0, 1), axis=0, copy=False)
# print(data_reshaped.shape)

## 4. Helping functions

In [ ]:
# function to plot and display the image
def plot_data(data):
  fig = plt.figure(figsize = (15, 10))
  plt.imshow(data, cmap = 'nipy_spectral')
  plt.colorbar()
  plt.axis('off')
  plt.show()

## 5. Dimensionality Reduction: Principle Componenet Analysis. 
- Reduce the number of model parameters
◦ Avoid over-fitting
◦ Reduce the computational load.

Data correlation and information redundancy.
Signal-noise ratio maximization.


 1. Subtract mean
 2. Calculate the covariance matrix
 3. Calculate eigenvectors and eigenvalues
of the covariance matrix
 4. Rank eigenvectors by its corresponding
eigenvalues
 4. Obtain P with its column vectors
corresponding to the top k eigenvectors

In [ ]:
# PCA
pca = PCA(n_components=data_array.shape[0])
components = pca.fit_transform(data_reshaped)
var_ratio = pca.explained_variance_ratio_
values = pca.singular_values_

# print(var_ratio.shape)
# print(values)

## 6. Optimal number of components:

In [ ]:
# function to calculate the wss loss to find out the optimal no of clusters
# for a give number of compenents
def calculate_WSS(points, kmax):
  sse = []
  for k in range(1, kmax+1):
    kmeans = cluster.KMeans(n_clusters = k).fit(points)
    centroids = kmeans.cluster_centers_
    pred_clusters = kmeans.predict(points)
    curr_sse = 0
    
    # calculate square of Euclidean distance of each point from its cluster center and add to current WSS
    for i in range(len(points)):
      curr_center = centroids[pred_clusters[i]]
      curr_sse += (points[i, 0] - curr_center[0]) ** 2 + (points[i, 1] - curr_center[1]) ** 2
      
    sse.append(curr_sse)
  return sse

kmax            = 10
components_num  = 5
sse = calculate_WSS(components[:,:components_num], kmax)
# plot the wss vs clusters 
plot_data(sse)